# Golden Section Search

In [1]:
import numpy as np
import typing
from numpy import typing as nptyping

In [6]:
def golden_section(f, boundary, maxiter=500, tol=1e-5):
    r = (-1 + np.sqrt(5)) / 2  # golden ratio
    a = boundary[0]  # lower bound
    b = boundary[1]  # upper bound

    for _ in range(maxiter):
        h = b - a
        if abs(h) < tol:
            return (a + b) / 2
        lower = b - r * h
        upper = a + r * h

        if f(lower) > f(upper):
            a = lower
        else:
            b = upper

    return (a + b) / 2

In [7]:
# Example of Golden Section Search
f = lambda x: np.exp(3 * x) + 5 * np.exp(-2 * x)
root = golden_section(f, [0, 3])
print(root)

0.2407956610080465


# Gradient Descent